In [1]:
import pandas as pd
import os
import numpy as np
pd.options.display.max_rows=300
pd.set_option('display.max_columns', None)# 列显示不全


In [2]:
# df_fluidin = pd.read_csv("/public/hanl/jupyter_dir/database/mimicIV20/raw/raw/icu/inputevents.csv",nrows=1)
# df_fluidin

# aumc

In [3]:
df_type = pd.read_csv("/public/hanl/jupyter_dir/kidney_sepsis_penotype_v3/02.AUMCdb_SAKI_trajCluster/df_mixAK_fea3_C3_aumc.csv")
df_type_filt = df_type.loc[:,["stay_id","groupHPD"]]
df_type_filt = df_type_filt.drop_duplicates()
all_lst = df_type_filt.stay_id.unique().tolist()
print("纳入患者数量：",len(all_lst))

df_type_filt["groupHPD"].value_counts()

纳入患者数量： 2183


2    1400
1     564
3     219
Name: groupHPD, dtype: int64

In [4]:
df_fluidin = pd.read_csv("/public/hanl/jupyter_dir/database/AMUCdb/raw/AmsterdamUMCdb-v1/drugitems.csv", encoding='ISO-8859-1')
df_fluidin =  df_fluidin[["admissionid","start","stop","fluidin"]]
df_fluidin.columns = ["stay_id","starttime","endtime","fluidin"]
df_fluidin["starttime"] = df_fluidin["starttime"]/1000/60/60
df_fluidin["endtime"] = df_fluidin["endtime"]/1000/60/60
# df_fluidin = df_fluidin[df_fluidin["fluidin"]!=0]
df_fluidin

,stay_id,starttime,endtime,fluidin
0,0,5.583333,5.600000,0.0
1,0,14.583333,14.600000,0.0
2,0,23.583333,23.600000,0.0
3,0,8.483333,9.483333,500.0
4,0,12.716667,13.716667,500.0
...,...,...,...,...
4907264,23552,18.950000,18.966667,0.0
4907265,23552,18.950000,18.966667,0.0
4907266,23552,4.250000,4.266667,0.0
4907267,23552,15.883333,15.900000,0.0


In [5]:
df_time = pd.read_csv("/public/hanl/jupyter_dir/kidney_sepsis_penotype_v3/00.data_aumc/disease_definition/AKI/aumcdb_sk_event_time.csv")
df_time = df_time[df_time.stay_id.isin(all_lst)]
df_time = df_time[["stay_id","saki_onset"]]

In [6]:
df_fb_fluid =  pd.merge(df_fluidin, df_time,how="inner",on="stay_id")
df_fb_fluid["time"] = (df_fb_fluid["starttime"] - df_fb_fluid["saki_onset"])/24
df_fb_fluid["time"] = df_fb_fluid["time"].map(lambda x:x//1) # -1指sepsis前6小时的数据，0指sepsis后第一个6小时的数据
df_fb_fluid["time"] = df_fb_fluid["time"].map(lambda x: x+1 if x>=0 else x)
df_fb_fluid = df_fb_fluid[(df_fb_fluid["time"]>=-2) &(df_fb_fluid["time"]<=7)] # 前七天与后7天
df_fb_fluid = df_fb_fluid.groupby(["stay_id", "time"]).agg({"fluidin":"sum"}).reset_index()
df_fb_fluid = df_fb_fluid.rename(columns={"fluidin":"colloid_bolus"})
df_fb_fluid.to_csv("aumc_fluidin.csv",index=False)

In [8]:
df_fb_fluid.shape

(13930, 3)

In [7]:
pwd

'/public/hanl/jupyter_dir/kidney_sepsis_penotype_v3/06.fluid_resuscitation/00.data'